In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-215126
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-215126


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "my-compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
max_nodes = 4

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, using it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 250, 1000), "--max_iter": choice(25, 50, 100, 200, 500)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                script='train.py', 
                compute_target=cluster_name,
                environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=30,
max_concurrent_runs=4
)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)



RunId: HD_d9715aa9-fad3-40d9-baaa-923094472c91
Web View: https://ml.azure.com/runs/HD_d9715aa9-fad3-40d9-baaa-923094472c91?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-215126/workspaces/quick-starts-ws-215126&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-11-12T18:49:31.598758][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-12T18:49:32.4589873Z][SCHEDULER][INFO]Scheduling job, id='HD_d9715aa9-fad3-40d9-baaa-923094472c91_0' 
[2022-11-12T18:49:32.5132252Z][SCHEDULER][INFO]Scheduling job, id='HD_d9715aa9-fad3-40d9-baaa-923094472c91_1' 
[2022-11-12T18:49:32.5492773Z][SCHEDULER][INFO]Scheduling job, id='HD_d9715aa9-fad3-40d9-baaa-923094472c91_2' 
[2022-11-12T18:49:32.6487705Z][SCHEDULER][INFO]Scheduling job, id='HD_d9715aa9-fad3-40d9-baaa-923094472c91_3' 
[2022-11-12T18:49:32.620246][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_d9715aa9-fad3-40d9-baaa-923094472c91',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-12T18:49:30.702607Z',
 'endTimeUtc': '2022-11-12T19:15:08.279798Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'c31ef61b-e362-4bfe-8106-b7aa0c34ddcc',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '40',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_d9715aa9-fad3-40d9-baaa-923094472c91_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d9715aa9-fad3-40d9-baaa-923094472c91_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'aml

In [21]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Details
print(best_run.get_details())

# metrics
print(best_run.get_metrics())

# files
print(best_run.get_file_names())

# Save model
# best_model = best_run.register_model(model_name='hyperdrive_model', model_path = 'outputs/')


{'runId': 'HD_d9715aa9-fad3-40d9-baaa-923094472c91_4', 'target': 'my-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2022-11-12T19:07:13.031098Z', 'endTimeUtc': '2022-11-12T19:07:33.581687Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': 'c31ef61b-e362-4bfe-8106-b7aa0c34ddcc', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '100', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'my-compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-env', 

In [17]:
best_model.download()

''

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)


In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

ModuleNotFoundError: No module named 'train'

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=False)
automl_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

best_automl_run, best_automl_model = automl_run.get_output()